# 1. Setup Mario

In [ ]:
!pip install gym_super_mario_bros==7.3.0 nes_py

In [ ]:
# Importa il gioco
import gym_super_mario_bros
# Importa il wrapper per il Joypad
from nes_py.wrappers import JoypadSpace
# Importa i controlli SEMPLIFICATI
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [ ]:
# Configura il gioco
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [ ]:
# Crea una flag - ripartire o no
done = True
# Cicla attraverso ogni frame del gioco
for step in range(100000): 
    # Avvia il gioco all'inizio
    if done: 
        # Riavvia il gioco
        env.reset()
    # Esegui azioni casuali
    state, reward, done, info = env.step(env.action_space.sample())
    # Mostra il gioco sullo schermo
    env.render()
# Chiudi il gioco
env.close()

# 2. Preprocess Environment

In [ ]:
# Installa pytorch
!pip install torch==1.10.1+cu113 torchvision==0.11.2+cu113 torchaudio===0.10.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [ ]:
# Installa Stable Baselines per strumenti di apprendimento per rinforzo
!pip install stable-baselines3[extra]

In [ ]:
# Importa il wrapper per l'accumulo dei frame e il wrapper per la conversione in scala di grigi
from gym.wrappers import GrayScaleObservation
# Importa i wrapper per la vettorizzazione
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Importa Matplotlib per mostrare l'impatto dell'accumulo dei frame
from matplotlib import pyplot as plt

In [ ]:
# 1. Crea l'ambiente base
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Semplifica i controlli
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Converti in scala di grigi
env = GrayScaleObservation(env, keep_dim=True)
# 4. Inserisci l'ambiente all'interno del Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Accumula i frame
env = VecFrameStack(env, 4, channels_order='last')

In [ ]:
state = env.reset()

In [ ]:
state, reward, done, info = env.step([5])

In [ ]:
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

# 3. Train the RL Model

In [ ]:
# Importa os per la gestione dei percorsi dei file
import os 
# Importa PPO per gli algoritmi
from stable_baselines3 import PPO
# Importa Base Callback per salvare i modelli
from stable_baselines3.common.callbacks import BaseCallback

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [ ]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [ ]:
# Configura il callback per il salvataggio del modello
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [ ]:
# Questo è il modello di intelligenza artificiale avviato
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.0000005, 
            n_steps=512)

In [ ]:
# Addestra il modello di intelligenza artificiale, qui è dove il modello inizia a imparare
model.learn(total_timesteps=4000000, callback=callback)

In [ ]:
model.save('thisisatestmodel')

# 4. Test it Out

In [ ]:
# Carica il modello
model = PPO.load('./train/best_model_4000000')

In [ ]:
state = env.reset()

In [ ]:
import warnings
warnings.filterwarnings("ignore")  # Sopprime i warning

# Contatori
episode = 0
victories = 0

# Traccia l'ultimo livello (mondo e stage)
last_world = 1
last_stage = 1

# Ciclo attraverso il gioco
while True: 
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()
    
    # Controlla se il gioco è terminato
    if done:
        episode += 1
        
        # Ottieni il mondo e lo stage attuali
        current_world = info[0].get("world", 1)
        current_stage = info[0].get("stage", 1)
        
        # Controlla se Mario ha completato un livello (cambiamento di mondo o stage)
        if (current_world > last_world) or (current_stage > last_stage):
            victories += 1
            print(f"Vittoria nell'episodio {episode}! Ora Mario è nel mondo {current_world}-{current_stage}.")
        
        # Aggiorna i valori di mondo e stage
        last_world = current_world
        last_stage = current_stage
        
        # Reimposta l'ambiente per il prossimo episodio
        state = env.reset()